In [ ]:
# Based on Ch. 3 example of "Deep Learning with Python" by F. Chollet
# Adds sklearn k-fold validation to the example

from keras.datasets import boston_housing
from sklearn.utils import shuffle
import numpy as np
import matplotlib.pyplot as plt
(train_data, train_targets), (test_data, test_targets) = boston_housing.load_data()

# Shuffle for kicks
train_data, train_targets = shuffle(train_data, train_targets, random_state = 0)
test_data, test_targets = shuffle(test_data, test_targets, random_state = 0)

# Targets to 2D tensors
train_targets = np.reshape(train_targets, (train_targets.shape[0], 1))
test_targets = np.reshape(test_targets, (test_targets.shape[0], 1))

In [ ]:
from sklearn.preprocessing import StandardScaler

sc_features = StandardScaler()
X_train = sc_features.fit_transform(train_data)
X_test = sc_features.transform(test_data)

sc_labels = StandardScaler()
sc_labels = sc_labels.fit(np.reshape(train_targets, (train_targets.shape[0], 1)))
Y_train = sc_labels.transform(train_targets)
Y_test = sc_labels.transform(test_targets)
Y_train = train_targets
Y_test = test_targets

In [ ]:
from keras import models, layers
from sklearn.model_selection import KFold

def build_model():
    model = models.Sequential()
    model.add(layers.Dense(64, activation = 'relu', input_shape = (train_data.shape[1], )))
    model.add(layers.Dense(64, activation = 'relu'))
    model.add(layers.Dense(1)) # 1D regression
    model.compile(optimizer = 'rmsprop', loss = 'mse', metrics = ['mse'])
    return model

def train_for(X_train, Y_train, X_validation, Y_validation):
    model = build_model()
    history = model.fit(X_train, Y_train, epochs = 500, verbose = 0, validation_data = (X_validation, Y_validation))
    return history

def k_fold_validate(folds):
    kf = KFold(n_splits=folds)
    split_indices = kf.split(X_train)
    histories = []
    for train_index, test_index in split_indices:
        X_train_split = X_train[train_index]
        Y_train_split = Y_train[train_index]
        X_test_split = X_train[test_index]
        Y_test_split = Y_train[test_index]
        history = train_for(X_train_split, Y_train_split, X_test_split, Y_test_split)
        histories.append(history)
    return histories
    
histories = k_fold_validate(5)


In [ ]:
def average_for(history_field):
    vals = [history.history[history_field] for history in histories]
    return np.mean(vals, axis = 0)

losses = average_for('loss')
val_losses = average_for('val_loss')
epochs = range(1, len(losses) + 1)

plt.clf()
plt.plot(epochs, losses, 'bo', label='Training loss')
plt.plot(epochs, val_losses, 'r-', label='Validation loss')
plt.title('Training and validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.xlim([10, len(epochs)])
plt.ylim([0, max(val_losses[10:])])
plt.legend()
plt.show()
